# 1. Setup

## 1.1 Installing Libraries

Reference: [Llama Index Installation and Setup](https://docs.llamaindex.ai/en/stable/getting_started/installation/)

In [ ]:
!pip install python-dotenv llama-index chromadb llama-index-vector-stores-chroma llama-index-retrievers-bm25 EbookLib html2text langchain-text-splitters

## 1.2 Importing Libraries

In [1]:
import chromadb

from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, PromptTemplate, get_response_synthesizer
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.ingestion import IngestionPipeline
from llama_index.llms.openai import OpenAI
from llama_index.core.retrievers import QueryFusionRetriever

from ebooklib import epub
import uuid
import os
from pathlib import Path
from dotenv import load_dotenv
import nest_asyncio
from enum import Enum

nest_asyncio.apply()

## 1.3 Importing Environment Variables

In [2]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

## 1.4 Setting up Embedding Model

In [3]:
embed_model = OpenAIEmbedding(api_key=OPENAI_API_KEY)

## 1.5 Setting up LLM

In [4]:
llm = OpenAI(api_key=OPENAI_API_KEY, model_name="gpt-4o-mini", temperature=0.1)

# 2. Setting up IndexedVectorStore

In [5]:
# The name "IndexedVectorStore" emphasizes that the class handles both the vector store and the index

class IndexedVectorStore:
    def __init__(self):
        self.db = chromadb.PersistentClient(path="./db")
        self.chroma_collection = self.db.get_or_create_collection("transcription_project")
        self.vector_store = ChromaVectorStore(chroma_collection=self.chroma_collection)
        self.index = VectorStoreIndex.from_vector_store(
            self.vector_store,
            embed_model=embed_model,
        )

    def add_documents(self, documents: list) -> None:
        # Add the documents to the LlamaIndex and persist them
        for document in documents:
            self.index.insert(document)
        self.index.storage_context.persist(persist_dir="./db")

In [6]:
vectorstore = IndexedVectorStore()

# 3. Loading Data from Directory using `SimpleDirectoryReader`

Reference: [Loaders](https://docs.llamaindex.ai/en/stable/understanding/loading/loading/)

Extracting Metadata Reference: [SimpleDirectoryReader](https://docs.llamaindex.ai/en/stable/module_guides/loading/simpledirectoryreader/)

We can specify a function that will read each file and extract metadata that gets attached to the resulting Document object for each file by passing the function as `file_metadata`

In [7]:
def extract_epub_metadata(book_path: str) -> dict:
    book_path = Path(book_path)
    if not book_path.exists():
        raise FileNotFoundError(f"EPUB file not found at path: {book_path}")
    book = epub.read_epub(str(book_path))

    return {
        "id": f"epub-{uuid.uuid4().hex}",
        "title": book.get_metadata("DC", "title")[0][0].rstrip(".epub") if book.get_metadata("DC", "title") else "N/A",
        "author": book.get_metadata("DC", "creator")[0][0] if book.get_metadata("DC", "creator") else "",
        "language": book.get_metadata("DC", "language")[0][0] if book.get_metadata("DC", "language") else "",
        "description": book.get_metadata("DC", "description")[0][0] if book.get_metadata("DC", "description") else "",
        "type": "epub",
        "embeddings": "openaiembeddings"
    }

In [8]:
documents = SimpleDirectoryReader(input_dir="./data", file_metadata=extract_epub_metadata).load_data()

e:\Capelin\llama-index-test\.venv\Lib\site-packages\ebooklib\epub.py:1395: UserWarning: In the future version we will turn default option ignore_ncx to True.
  warnings.warn('In the future version we will turn default option ignore_ncx to True.')
e:\Capelin\llama-index-test\.venv\Lib\site-packages\ebooklib\epub.py:1423: FutureWarning: This search incorrectly ignores the root element, and will be fixed in a future version.  If you rely on the current behaviour, change it to './/xmlns:rootfile[@media-type]'
  for root_file in tree.findall('//xmlns:rootfile[@media-type]', namespaces={'xmlns': NAMESPACES['CONTAINERNS']}):


In [9]:
print(f"Total Documents: {len(documents)}")

Total Documents: 1


In [10]:
print(documents[0].metadata)

{'id': 'epub-24542662e8fa47f2b48246bcd9c6c2e9', 'title': "Theological Instructions (Amuzish-e Aqa'id)", 'author': 'Muhammad Taqi Misbah Yazdi', 'language': 'en', 'description': '', 'type': 'epub', 'embeddings': 'openaiembeddings'}


Loading a new book

In [15]:
new_book = SimpleDirectoryReader(input_files=["./data/give_and_take.epub"], file_metadata=extract_epub_metadata).load_data()
print(f"Metadata of first element: {new_book[0].metadata}")

# This way, we can load a new book and can use the same VectorStore object to add the new book to the index

Metadata of first element: {'id': 'epub-6d7d7179a80e414c9c18e6715cae457f', 'title': 'Give and Tak', 'author': 'Unknown', 'language': 'en', 'description': '', 'type': 'epub'}


# 4. Transforming

After the data is loaded, you then need to process and transform your data before putting it into a storage system. These transformations include chunking, extracting metadata, and embedding each chunk. This is necessary to make sure that the data can be retrieved, and used optimally by the LLM.

An `IngestionPipeline` uses a concept of Transformations that are applied to input data. These Transformations are applied to your input data

Reference: [IngestionPipeline](https://docs.llamaindex.ai/en/stable/module_guides/loading/ingestion_pipeline/)

In [11]:
chunk_size = 512
overlap_percentage = 0.25
chunk_overlap = int(chunk_size * overlap_percentage)

print(f"Chunk Size: {chunk_size}, Overlap Percentage: {overlap_percentage}, Chunk Overlap: {chunk_overlap}")

Chunk Size: 512, Overlap Percentage: 0.25, Chunk Overlap: 128


In [12]:
pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap, paragraph_separator="\n\n\n"),
        embed_model # OpenAIEmbedding
    ],
    vector_store=vectorstore.vector_store,
)

In [13]:
documents = pipeline.run(documents=documents)

In [14]:
print(f"Total Documents: {len(documents)}")

Total Documents: 516


In [15]:
for i, doc in enumerate(documents[:10]):
    print(f"Document {i + 1} Length: {len(doc.text)}")

Document 1 Length: 1957
Document 2 Length: 2062
Document 3 Length: 2055
Document 4 Length: 2015
Document 5 Length: 2221
Document 6 Length: 2143
Document 7 Length: 1927
Document 8 Length: 1398
Document 9 Length: 1524
Document 10 Length: 1553


Function to load and add a new book to vectorstore

In [18]:
def add_book(book_path: str):
    print(f"Loading book from path: {book_path}")
    new_book = SimpleDirectoryReader(input_files=[book_path], file_metadata=extract_epub_metadata).load_data()
    print(f"Loaded book with metadata: {new_book[0].metadata}")
    new_book = pipeline.run(documents=new_book)
    print("Book added successfully!")

In [19]:
add_book("./data/give_and_take.epub")

Loading book from path: ./data/give_and_take.epub


e:\Capelin\llama-index-test\.venv\Lib\site-packages\ebooklib\epub.py:1395: UserWarning: In the future version we will turn default option ignore_ncx to True.
  warnings.warn('In the future version we will turn default option ignore_ncx to True.')
e:\Capelin\llama-index-test\.venv\Lib\site-packages\ebooklib\epub.py:1423: FutureWarning: This search incorrectly ignores the root element, and will be fixed in a future version.  If you rely on the current behaviour, change it to './/xmlns:rootfile[@media-type]'
  for root_file in tree.findall('//xmlns:rootfile[@media-type]', namespaces={'xmlns': NAMESPACES['CONTAINERNS']}):


Loaded book with metadata: {'id': 'epub-b09eb4f758f54495a29b6b23763dffb3', 'title': 'Give and Tak', 'author': 'Unknown', 'language': 'en', 'description': '', 'type': 'epub'}
Book added successfully!


# 5. Performing Vector Search

In [7]:
query_gen_prompt = """You are an AI language model assistant specializing in query expansion. Your task is to generate {num_queries} diverse versions of the given user question. These variations will be used to retrieve relevant documents from a vector database, helping to overcome limitations of distance-based similarity search.

Original question: {query}

Instructions:
1. Create {num_queries} unique variations of the original question.
2. Ensure each variation maintains the core intent of the original question.
3. Use different phrasings, synonyms, or perspectives for each variation.
4. Consider potential context or implications not explicitly stated in the original question.
5. Avoid introducing new topics or drastically changing the meaning of the question.

Please provide your {num_queries} question variations, each on a new line:
"""

In [8]:
top_n = 5
num_queries = 5
question = "Why is there only one God?"

vector_retriever = vectorstore.index.as_retriever(similarity_top_k=top_n)

In [9]:
retriever = QueryFusionRetriever(
    [vector_retriever],
    similarity_top_k=top_n,
    num_queries=num_queries,  # set this to 1 to disable query generation
    mode="reciprocal_rerank",
    use_async=True,
    verbose=True,
    query_gen_prompt=query_gen_prompt,
)

In [10]:
# it will genereate n - 1 queries since the original query is also included
nodes_with_scores = retriever.retrieve(question)

Generated queries:
1. What is the reason behind the belief in a singular God?
2. How do monotheistic religions explain the existence of only one God?
3. What factors contribute to the concept of a solitary deity in various faiths?
4. Is there a specific rationale for the monotheistic principle of a singular divine being?


In [11]:
# Printing first document
print("------- BOOK INFO -------")
print(f"Book Title: {nodes_with_scores[0].metadata['title']}")
print(f"Book ID   : {nodes_with_scores[0].metadata['id']}")
print(f"Author    : {nodes_with_scores[0].metadata['author']}")

print("\n------- TEXT -------")
print(nodes_with_scores[0].text)

------- BOOK INFO -------
Book Title: Theological Instructions (Amuzish-e Aqa'id)
Book ID   : epub-24542662e8fa47f2b48246bcd9c6c2e9
Author    : Muhammad Taqi Misbah Yazdi

------- TEXT -------
5- Why can one not regard every group of creation to be created by its
particular god?

6- How would you criticise the assumption that the entire universe is created
by one God, and at the same time claim that there are many lords and
executers?

7- From where does the illusion of several gods come from, and how can it be
nullified?

8- Why does it make no difference to the oneness of lordship and creatorship,
if one believes in the Divine authority for the saints?

# Lesson 17: The Meaning Of The Oneness Of The God

## Introduction

The word monotheism from the lexical point of view means ‘oneness or unity’,
but in philosophy, scholastic theology, ethics _(akhlaq),_ and mysticism it
has been applied with various different meanings. However it gives the same
basic meaning in all of these sciences

# 7. Generate Answer

In [12]:
answer_template = """You are a knowledgeable AI assistant tasked with answering questions based on the provided context. Your goal is to provide a comprehensive, accurate, and well-structured response using Chain-of-Thought reasoning.

Context:
{context_str}

Question: {query_str}

Instructions:
1. Carefully analyze the given context and question.
2. Use Chain-of-Thought reasoning to break down your answer into clear steps:
   a. First, identify the key components of the question, such as sub-problems that need to be explained before an answer can be derived
   b. Then, for each component, explain your thought process as you analyze the relevant information from the context.
   c. Show how you're connecting different pieces of information to form your conclusion.
3. Provide a detailed answer using only the information from the context.
4. If the context doesn't contain enough information to fully answer the question, state this clearly and explain why.
5. Organize your response with appropriate headings and subheadings for clarity.
6. Use bullet points or numbered lists where applicable to improve readability.
7. If relevant, include brief examples or analogies to illustrate key points.
8. After your detailed Chain-of-Thought reasoning, summarize your main points at the end of the response.
9. At the end, list all the contexts used in your reasoning. After your response, add a "References" section where you list the full contexts that you used arrive at your answer. Provide as much detail as available from each context (e.g., book title, author, full text of the relevant contexts. For video sources, include the url to the video.

For the references, use the format:

# References:
(for each context:)
## Context Id: title
Context excerpt (print as it is)

Please format your entire response in markdown for optimal readability.
"""

answer_prompt = PromptTemplate(answer_template)
# answer_prompt = BasePromptTemplate(answer_template)

## Response Modes


In [13]:
class ResponseMode(str, Enum):
    """Response modes of the response builder (and synthesizer)."""

    REFINE = "refine"
    """
    Refine is an iterative way of generating a response.
    We first use the context in the first node, along with the query, to generate an \
    initial answer.
    We then pass this answer, the query, and the context of the second node as input \
    into a “refine prompt” to generate a refined answer. We refine through N-1 nodes, \
    where N is the total number of nodes.
    """

    COMPACT = "compact"
    """
    Compact and refine mode first combine text chunks into larger consolidated chunks \
    that more fully utilize the available context window, then refine answers \
    across them.
    This mode is faster than refine since we make fewer calls to the LLM.
    """

    SIMPLE_SUMMARIZE = "simple_summarize"
    """
    Merge all text chunks into one, and make a LLM call.
    This will fail if the merged text chunk exceeds the context window size.
    """

    TREE_SUMMARIZE = "tree_summarize"
    """
    Build a tree index over the set of candidate nodes, with a summary prompt seeded \
    with the query.
    The tree is built in a bottoms-up fashion, and in the end the root node is \
    returned as the response
    """

    GENERATION = "generation"
    """Ignore context, just use LLM to generate a response."""

    NO_TEXT = "no_text"
    """Return the retrieved context nodes, without synthesizing a final response."""

    CONTEXT_ONLY = "context_only"
    """Returns a concatenated string of all text chunks."""

    ACCUMULATE = "accumulate"
    """Synthesize a response for each text chunk, and then return the concatenation."""

    COMPACT_ACCUMULATE = "compact_accumulate"
    """
    Compact and accumulate mode first combine text chunks into larger consolidated \
    chunks that more fully utilize the available context window, then accumulate \
    answers for each of them and finally return the concatenation.
    This mode is faster than accumulate since we make fewer calls to the LLM.
    """

In [14]:
def generate_answer(question, documents, answer_prompt, response_mode, verbose):
    response_synthesizer = get_response_synthesizer(response_mode=response_mode, llm=llm, text_qa_template=answer_prompt, verbose=verbose)

    response = response_synthesizer.synthesize(
        question, nodes=documents
    )

    return response

`REFINE`:

Refine is an iterative way of generating a response. We first use the context in the first node, along with the query, to generate an initial answer. We then pass this answer, the query, and the context of the second node as input into a “refine prompt” to generate a refined answer. We refine through N-1 nodes, where N is the total number of nodes.

In [37]:
answer_md = generate_answer(question=question, documents=nodes_with_scores, answer_prompt=answer_prompt, response_mode=ResponseMode.REFINE, verbose=True)

[INFO] Query: Why is there only one God?
----------
[INFO] Processing chunk 1/5
[INFO] Initial response for chunk 1
[INFO] QUESTION SENT TO LLM: metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>} template_vars=['context_str', 'query_str'] kwargs={'query_str': 'Why is there only one God?'} output_parser=None template_var_mappings=None function_mappings=None template='You are a knowledgeable AI assistant tasked with answering questions based on the provided context. Your goal is to provide a comprehensive, accurate, and well-structured response using Chain-of-Thought reasoning.\n\nContext:\n{context_str}\n\nQuestion: {query_str}\n\nInstructions:\n1. Carefully analyze the given context and question.\n2. Use Chain-of-Thought reasoning to break down your answer into clear steps:\n   a. First, identify the key components of the question, such as sub-problems that need to be explained before an answer can be derived\n   b. Then, for each component, explain your thought process as you ana

In [27]:
print(answer_md)

The belief in multiple gods creating and managing the universe is logically refuted by the unity, coherence, and interdependence observed in the universe. This refutation highlights the contradictions and complexities that arise from such a belief, reinforcing the concept of a singular, supreme creator.


`COMPACT`:

Compact and refine mode first combine text chunks into larger consolidated chunks that more fully utilize the available context window, then refine answers across them. This mode is faster than refine since we make fewer calls to the LLM.

In [28]:
answer_md = generate_answer(question=question, documents=nodes_with_scores, answer_prompt=answer_prompt, response_mode=ResponseMode.COMPACT, verbose=True)

[INFO] Query: Why is there only one God?
----------
[INFO] Processing chunk 1/1
[INFO] Initial response for chunk 1
[INFO] QUESTION SENT TO LLM: metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>} template_vars=['context_str', 'query_str'] kwargs={'query_str': 'Why is there only one God?'} output_parser=None template_var_mappings=None function_mappings=None template='You are a knowledgeable AI assistant tasked with answering questions based on the provided context. Your goal is to provide a comprehensive, accurate, and well-structured response using Chain-of-Thought reasoning.\n\nContext:\n{context_str}\n\nQuestion: {query_str}\n\nInstructions:\n1. Carefully analyze the given context and question.\n2. Use Chain-of-Thought reasoning to break down your answer into clear steps:\n   a. First, identify the key components of the question, such as sub-problems that need to be explained before an answer can be derived\n   b. Then, for each component, explain your thought process as you ana

In [29]:
print(answer_md)

# Answer:

## Key Components:
1. Explanation of the oneness of God.
2. Critique of polytheistic beliefs.
3. Refutation of the assumption of multiple gods.
4. Understanding the illusion of several gods.
5. Impact of Divine authority on the oneness of lordship and creatorship.

## Chain-of-Thought Reasoning:

### 1. Explanation of the Oneness of God:
- The context emphasizes the oneness of God as a fundamental belief.
- Monotheism entails believing in the unity of God and negating plurality and multiplicity.
- Monotheism rejects the existence of two or more independent gods.
- The essence of God is considered self-evident and not composed of parts.

### 2. Critique of Polytheistic Beliefs:
- Polytheistic beliefs involve attributing lordship to entities other than God.
- Polytheists may acknowledge the Creator but assign independent administrators to the world.
- The foundation of polytheism lies in recognizing other gods alongside the Supreme Creator.
- The Noble Qur’an highlights strugg

`SIMPLE_SUMMARIZE`:

Merge all text chunks into one, and make a LLM call. This will fail if the merged text chunk exceeds the context window size.

In [30]:
answer_md = generate_answer(question=question, documents=nodes_with_scores, answer_prompt=answer_prompt, response_mode=ResponseMode.SIMPLE_SUMMARIZE, verbose=True)

[INFO] Received query: 'Why is there only one God?'
[INFO] Question sent to LLM: metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>} template_vars=['context_str', 'query_str'] kwargs={'query_str': 'Why is there only one God?'} output_parser=None template_var_mappings=None function_mappings=None template='You are a knowledgeable AI assistant tasked with answering questions based on the provided context. Your goal is to provide a comprehensive, accurate, and well-structured response using Chain-of-Thought reasoning.\n\nContext:\n{context_str}\n\nQuestion: {query_str}\n\nInstructions:\n1. Carefully analyze the given context and question.\n2. Use Chain-of-Thought reasoning to break down your answer into clear steps:\n   a. First, identify the key components of the question, such as sub-problems that need to be explained before an answer can be derived\n   b. Then, for each component, explain your thought process as you analyze the relevant information from the context.\n   c. Show how 

In [31]:
print(answer_md)

# Answer:

## Key Components:
1. **Why is there only one God?**
   - Explanation of monotheism and the oneness of God.
   - Critique of polytheistic beliefs.
   - Arguments against the existence of multiple gods.

## Chain-of-Thought Reasoning:

### Explanation of Monotheism and Oneness of God:
1. **Monotheism Definition:**
   - Monotheism refers to the belief in the existence of only one God.
   - It involves the negation of plurality and multiplicity in the essence of God.
   - Monotheism opposes polytheism, which believes in multiple independent gods.

2. **Oneness of God:**
   - The oneness of God is a fundamental concept in monotheistic religions.
   - It signifies that there is only one supreme being who is the creator and lord of the universe.
   - Monotheism emphasizes the unity and uniqueness of God's essence.

### Critique of Polytheistic Beliefs:
1. **Polytheistic Beliefs:**
   - Polytheism involves the belief in multiple gods, each with independent powers and authorities.
 

`TREE_SUMMARIZE`:

Build a tree index over the set of candidate nodes, with a summary prompt seeded with the query. The tree is built in a bottoms-up fashion, and in the end the root node is returned as the response

In [20]:
from llama_index.core.response_synthesizers import TreeSummarize

summarizer = TreeSummarize(verbose=True, summary_template=answer_prompt, llm=llm)

response = summarizer.get_response(question, str(nodes_with_scores))

[INFO] Received query: 'Why is there only one God?'
[INFO] Formatted summary template: 'metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>} template_vars=['context_str', 'query_str'] kwargs={'query_str': 'Why is there only one God?'} output_parser=None template_var_mappings=None function_mappings=None template='You are a knowledgeable AI assistant tasked with answering questions based on the provided context. Your goal is to provide a comprehensive, accurate, and well-structured response using Chain-of-Thought reasoning.\n\nContext:\n{context_str}\n\nQuestion: {query_str}\n\nInstructions:\n1. Carefully analyze the given context and question.\n2. Use Chain-of-Thought reasoning to break down your answer into clear steps:\n   a. First, identify the key components of the question, such as sub-problems that need to be explained before an answer can be derived\n   b. Then, for each component, explain your thought process as you analyze the relevant information from the context.\n   c. Sh

In [22]:
print(response)

# Answer:

## Key Components:
1. **Monotheistic Beliefs**: Understanding the core tenets of monotheistic religions.
2. **Reasoning for One God**: Exploring the rationale behind the belief in a singular deity.

## Chain-of-Thought Reasoning:

### 1. Monotheistic Beliefs:
- **Historical Context**: Monotheistic religions like Judaism, Christianity, and Islam have long upheld the belief in one God.
- **Scriptural Basis**: Texts such as the Bible and the Quran explicitly state the oneness of God and prohibit worship of multiple deities.
- **Central Tenet**: Monotheism emphasizes the uniqueness and supremacy of a single divine being.

### 2. Reasoning for One God:
- **Unity of Creation**: Monotheistic faiths argue that a singular God is the source of creation, implying a unified origin for all existence.
- **Logical Consistency**: Belief in one God provides a coherent worldview, avoiding conflicts and contradictions that may arise from multiple deities.
- **Supremacy and Sovereignty**: One G

`GENERATION`:

Ignore context, just use LLM to generate a response.

In [22]:
answer_md = generate_answer(question=question, documents=nodes_with_scores, answer_prompt=answer_prompt, response_mode=ResponseMode.GENERATION, verbose=True)

[INFO] Received query: 'Why is there only one God?'
[INFO] Input prompt being used: 'metadata={'prompt_type': <PromptType.SIMPLE_INPUT: 'simple_input'>} template_vars=['query_str'] kwargs={} output_parser=None template_var_mappings=None function_mappings=None template='{query_str}''
[INFO] Using predict mode (non-streaming).
[INFO] Response from LLM: 'In monotheistic religions such as Christianity, Islam, and Judaism, there is only one God because it is believed that there is only one ultimate, supreme being who is all-powerful, all-knowing, and all-loving. This belief is based on the idea that there can only be one source of ultimate truth, goodness, and power in the universe. Additionally, monotheistic religions teach that God is indivisible and cannot be divided into multiple beings.'


In [23]:
print(answer_md)

In monotheistic religions such as Christianity, Islam, and Judaism, there is only one God because it is believed that there is only one ultimate, supreme being who is all-powerful, all-knowing, and all-loving. This belief is based on the idea that there can only be one source of ultimate truth, goodness, and power in the universe. Additionally, monotheistic religions teach that God is indivisible and cannot be divided into multiple beings.


`NO_TEXT`:

Return the retrieved context nodes, without synthesizing a final response.

In [29]:
answer_md = generate_answer(question=question, documents=nodes_with_scores, answer_prompt=answer_prompt, response_mode=ResponseMode.NO_TEXT, verbose=True)
print(answer_md) # this will return nothing since the response mode is set to NO_TEXT

None


In [24]:
print(answer_md.source_nodes) # But this will return the source nodes

[NodeWithScore(node=TextNode(id_='3588e51f-2d98-479d-ac3e-64276f1f8963', embedding=None, metadata={'id': 'epub-24542662e8fa47f2b48246bcd9c6c2e9', 'title': "Theological Instructions (Amuzish-e Aqa'id)", 'author': 'Muhammad Taqi Misbah Yazdi', 'language': 'en', 'description': '', 'type': 'epub', 'embeddings': 'openaiembeddings'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='a68c1e50-47bc-467a-9589-0a84bf4d0357', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'id': 'epub-24542662e8fa47f2b48246bcd9c6c2e9', 'title': "Theological Instructions (Amuzish-e Aqa'id)", 'author': 'Muhammad Taqi Misbah Yazdi', 'language': 'en', 'description': '', 'type': 'epub', 'embeddings': 'openaiembeddings'}, hash='c31b0860b0e4f8

`CONTEXT_ONLY`:

Returns a concatenated string of all text chunks.

In [30]:
answer_md = generate_answer(question=question, documents=nodes_with_scores, answer_prompt=answer_prompt, response_mode=ResponseMode.CONTEXT_ONLY, verbose=True)
print(answer_md)

id: epub-24542662e8fa47f2b48246bcd9c6c2e9
title: Theological Instructions (Amuzish-e Aqa'id)
author: Muhammad Taqi Misbah Yazdi
language: en
description: 
type: epub
embeddings: openaiembeddings

There is no power except by Allah!” (Holy
Qur’an,[18:39](/printepub/book/export/html/123734#quran_ref_189052)).** _

### D. The Two Important Results Achieved

The result of the unity of Divine action is that nothing other than God
deserves worship, because as we have indicated before, a being does not
deserve to be worshipped by just being a creator or a lord. In other words,
Divinity _(uluhiyyah)_ is the necessary condition of lordship and creatorship.

From another angle, the result of monotheism in the latter meaning is that the
entirety of human reliance must be upon God, and in all of works He must be
trusted and solely from Him help must be requested. Man’s fear and hope ought
to be from Him, and when the sources for the completion of needs are out of
reach, one must not despair, becaus

`ACCUMULATE`:

Synthesize a response for each text chunk, and then return the concatenation.

In [15]:
answer_md = generate_answer(question=question, documents=nodes_with_scores, answer_prompt=answer_prompt, response_mode=ResponseMode.ACCUMULATE, verbose=True)

[INFO] Query: 'Why is there only one God?'
[INFO] Text chunks received: 5 chunks
[INFO] Generated text QA template: 'metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>} template_vars=['context_str', 'query_str'] kwargs={'query_str': 'Why is there only one God?'} output_parser=None template_var_mappings=None function_mappings=None template='You are a knowledgeable AI assistant tasked with answering questions based on the provided context. Your goal is to provide a comprehensive, accurate, and well-structured response using Chain-of-Thought reasoning.\n\nContext:\n{context_str}\n\nQuestion: {query_str}\n\nInstructions:\n1. Carefully analyze the given context and question.\n2. Use Chain-of-Thought reasoning to break down your answer into clear steps:\n   a. First, identify the key components of the question, such as sub-problems that need to be explained before an answer can be derived\n   b. Then, for each component, explain your thought process as you analyze the relevant informatio

In [16]:
print(answer_md)

Response 1: # Answer:

## Key Components of the Question:
1. Why is there only one God?

## Chain-of-Thought Reasoning:

### 1. The Negation of Plurality:
- The initial meaning of monotheism is believing in the unity of God and negating plurality and multiplicity outside His essence.
- This belief opposes polytheism, which involves belief in multiple independent gods.
- Monotheism asserts that there is only one God, and all other entities are not divine beings.

### 2. The Negation of Composition:
- Another aspect of monotheism is the belief in the oneness of God's essence, which is self-evident and not composed of parts.
- This concept rejects the idea of God being made up of actual or potential parts.
- Monotheism emphasizes the simplicity and indivisibility of God's essence.

### 3. Unity of Lordship and Creatorship:
- Belief in the oneness of God also entails the unity of lordship and creatorship.
- Even if one believes in the Divine authority of saints or intermediaries, it does n

`COMPACT_ACCUMULATE`:

Compact and accumulate mode first combine text chunks into larger consolidated chunks that more fully utilize the available context window, then accumulate answers for each of them and finally return the concatenation. This mode is faster than accumulate since we make fewer calls to the LLM.

In [17]:
answer_md = generate_answer(question=question, documents=nodes_with_scores, answer_prompt=answer_prompt, response_mode=ResponseMode.COMPACT_ACCUMULATE, verbose=True)

[INFO] Query: 'Why is there only one God?'
[INFO] Text chunks received: 1 chunks
[INFO] Generated text QA template: 'metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>} template_vars=['context_str', 'query_str'] kwargs={'query_str': 'Why is there only one God?'} output_parser=None template_var_mappings=None function_mappings=None template='You are a knowledgeable AI assistant tasked with answering questions based on the provided context. Your goal is to provide a comprehensive, accurate, and well-structured response using Chain-of-Thought reasoning.\n\nContext:\n{context_str}\n\nQuestion: {query_str}\n\nInstructions:\n1. Carefully analyze the given context and question.\n2. Use Chain-of-Thought reasoning to break down your answer into clear steps:\n   a. First, identify the key components of the question, such as sub-problems that need to be explained before an answer can be derived\n   b. Then, for each component, explain your thought process as you analyze the relevant informatio

In [18]:
print(answer_md)

Response 1: # Answer:

## Key Components:
1. **Monotheism vs. Polytheism**: Understanding the difference between monotheism (belief in one God) and polytheism (belief in multiple gods).
2. **Arguments for Monotheism**: Exploring the arguments presented in the context that support the belief in one God.
3. **Unity of Lordship and Creatorship**: Examining how the oneness of God is connected to the unity of lordship and creatorship.
4. **Critique of Polytheistic Beliefs**: Analyzing why the assumption of multiple gods is not logically feasible.

## Chain-of-Thought Reasoning:

### 1. Monotheism vs. Polytheism:
- **Monotheism**: The belief in one God, which is the core principle of monotheistic religions like Islam.
- **Polytheism**: The belief in multiple gods, which is contrary to monotheistic beliefs.

### 2. Arguments for Monotheism:
- **Negation of Plurality**: Monotheism involves believing in the unity of God and rejecting the idea of multiple independent gods.
- **Negation of Compos